In [14]:
import cv2
import time

In [15]:
def apply_brightness_contrast(input_img, brightness=0, contrast=0):
    if brightness != 0:
        if brightness > 0:
            shadow = brightness
            highlight = 255
        else:
            shadow = 0
            highlight = 255 + brightness
        alpha_b = (highlight - shadow) / 255
        gamma_b = shadow

        input_img = cv2.addWeighted(input_img, alpha_b, input_img, 0, gamma_b)

    if contrast != 0:
        f = 131 * (contrast + 127) / (127 * (131 - contrast))
        alpha_c = f
        gamma_c = 127 * (1 - f)

        input_img = cv2.addWeighted(input_img, alpha_c, input_img, 0, gamma_c)

    return input_img

In [16]:
cap = cv2.VideoCapture('rtsp://210.99.70.120:1935/live/cctv009.stream')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('save_path', fourcc, 20.0, (640, 480))
is_recording = False
start_time = None
brightness = 0
contrast = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("카메라에서 프레임을 읽을 수 없습니다.")
        break

    frame = apply_brightness_contrast(frame, brightness, contrast)

    if is_recording:
        if start_time is None:
            start_time = time.time()
        elapsed_time = time.time() - start_time
        minutes = int(elapsed_time // 60)
        seconds = int(elapsed_time % 60)

        out.write(frame)
        cv2.circle(frame, (35, 35), 13, (0, 0, 255), -1)

        time_text = f"Recording: {minutes:02}:{seconds:02}"
        cv2.putText(frame, time_text, (60, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    cv2.imshow('Video Recorder', frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord(' '): #space
        is_recording = not is_recording

    # 'w'와 's'로 밝기 조절
    if key == ord('w'):
        brightness = min(255, brightness + 10)
    elif key == ord('s'):
        brightness = max(-255, brightness - 10)

    # 'a'와 'd'로 대비 조절
    if key == ord('a'):
        contrast = max(-127, contrast - 10)
    elif key == ord('d'):
        contrast = min(127, contrast + 10)


    if key == 27: 
        break

cap.release()
out.release()
cv2.destroyAllWindows()